In [17]:
import pandas as pd
import numpy as np
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [120]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)

In [121]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": np.nan, "Wait-Listed": np.nan, "Accepted": 1, "Will Attend": 1}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3, np.nan: np.nan}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [122]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['status'] = df.apply(map_statuses, axis=1)
df['hs_type'] = df.apply(map_hs_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
df['tuition'] = df.apply(get_tuition, axis=1)

c0 = .5
c1 = .4
c2 = .1
df.['split_metric'] = c0*df['test_score'] + c1*df['gpa_uw'] + c2*(1-df['class_rank'])

for col in [',in_state_tuition',',out_state_tuition','cost_attendance','school', ',city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', ',gpa_w']:
    try:
        df = df.drop(col, axis=1)
    except:
        pass
    
df = df.dropna()
replace_columns = ['hs_state', 'state', 'gender', 'city']
for col in replace_columns:
    try:
        df[col].replace(get_type_map(df[col]), inplace=True)
    except:
        pass

In [158]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import precision_recall_fscore_support as score

k = 1
df0 = df[df.status == 0]
df2 = df[df.status == 1].sample(n = int(k*len(df0)))
df_final = pd.concat([df0, df2])
df_final.sort_values(by=['split_metric'])
X = df_final.drop('status', axis=1)
X = X.drop('split_metric', axis=1)
y = df_final['status']
    
mlps = []
clusters = 5
for i in range(0,len(X),len(X)/5):
    X_train, X_test, y_train, y_test = train_test_split(X[i:i+len(X)/5], y[i:i+len(X)/5])

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(8,8,8),max_iter=1000)
    mlp.fit(X_train,y_train)
    pred = mlp.predict(X_test)
    mlps.append(score(y_test,pred))

# logisticRegr = LogisticRegression()
# logisticRegr.fit(X_train, y_train)
# pred = logisticRegr.predict(X_test)
# logs.append(score(y_test,pred))

# ridgeRegr = RidgeClassifier()
# ridgeRegr.fit(X_train, y_train)
# pred = ridgeRegr.predict(X_test)
# rids.append(score(y_test,pred))

In [168]:
totals = (sum([x[3][0] for x in mlps]),sum(x[3][1] for x in mlps))
sums = [([x[0][0]*x[3][0], x[0][1]*x[3][1]],[x[1][0]*x[3][0], x[1][1]*x[3][1]],[x[2][0]*x[3][0], x[2][1]*x[3][1]]) for x in mlps]
sums = ((sum([x[0][0] for x in sums])+sum([x[0][1] for x in sums]))/sum(totals),
        (sum([x[1][0] for x in sums])+sum([x[1][1] for x in sums]))/sum(totals),
        (sum([x[2][0] for x in sums])+sum([x[2][1] for x in sums]))/sum(totals))
print(sums)

(0.8492339444674628, 0.8486672710788757, 0.8486290091260691)
(0.8398154600964157, 0.8381368993653672, 0.8379660180650446)
(0.8334930871447289, 0.8299138712601994, 0.8294948514558331)
